In [16]:
import os
import sys
import numpy as np
import pandas as pd
import librosa
import librosa.display
import soundfile as sf
import matplotlib.pyplot as plt
import IPython.display as ipd

In [17]:
# diretório de entrada
DIR_ENTRADA = 'mp3'

# diretório de saída
DIR_SAIDA = 'wavbal5'

# duração de cada trecho de áudio
DURACAO_TRECHO = 5 # segundos
SAMPLE_RATE = 16000 # 16 kHz

# calcular a quantidade de pontos na onda
PONTOS_ONDA = SAMPLE_RATE * DURACAO_TRECHO # 16 kHz * 5 s = 80000

## Verificar espécies disponíveis

In [18]:
especies = os.listdir(DIR_ENTRADA)
#especies

In [19]:
arquivos = []
for especie in especies:
    arquivos.append(len(os.listdir(DIR_ENTRADA + '/' + especie)))
#arquivos

In [20]:
df = pd.DataFrame({
    'especie': especies,
    'arquivos_mp3': arquivos
})
df

,especie,arquivos_mp3
0,mimus_saturninus,148
1,ramphocelus_carbo,185
2,columba_livia,40
3,turdus_leucomelas,322
4,caracara_plancus,90
5,megarynchus_pitangua,401
6,vanellus_chilensis,280
7,pitangus_sulphuratus,556
8,turdus_rufiventris,209
9,furnarius_rufus,152


## Recriar diretórios de saída

In [6]:
# excluir diretórios de saída

import shutil
shutil.rmtree(DIR_SAIDA)
print(DIR_SAIDA)

wavbal5


In [7]:
# criar diretórios de saída

if not os.path.isdir(DIR_SAIDA):
    os.mkdir(DIR_SAIDA)
print(DIR_SAIDA)

for especie in especies:
    dir_especie = os.path.join(DIR_SAIDA, especie)
    if not os.path.isdir(dir_especie):
        os.mkdir(dir_especie)
    print(dir_especie)

wavbal5
wavbal5/mimus_saturninus
wavbal5/ramphocelus_carbo
wavbal5/columba_livia
wavbal5/turdus_leucomelas
wavbal5/caracara_plancus
wavbal5/megarynchus_pitangua
wavbal5/vanellus_chilensis
wavbal5/pitangus_sulphuratus
wavbal5/turdus_rufiventris
wavbal5/furnarius_rufus
wavbal5/theristicus_caudatus


## Processar arquivos de áudio

In [11]:
import warnings
warnings.filterwarnings('ignore')

In [12]:
%%time

for especie in especies:
    print("especie:", especie)
    
    for arquivo in os.listdir(os.path.join(DIR_ENTRADA, especie)):
#    for arquivo in os.listdir(os.path.join(DIR_ENTRADA, especie))[:20]:

        # montar nome do arquivo de entrada
        entrada = os.path.join(DIR_ENTRADA, especie, arquivo)
        if not entrada.endswith('.mp3'):
            continue
        #print("entrada:", entrada)

        prefixo_saida = os.path.join(DIR_SAIDA, especie, arquivo.replace('.mp3', '-'))
        #print("prefixo_saida:", prefixo_saida)
        
        # carregar áudio original em formato MP3
        y, sr = librosa.load(entrada, sr=SAMPLE_RATE, mono=True)

        # remover silêncio nas extremidades do áudio
        yt, index = librosa.effects.trim(y)
        duracao = yt.shape[0] / sr
        
        # calcular o número de cortes a serem efetuados no áudio
        numero_cortes = round(duracao / DURACAO_TRECHO)
        if numero_cortes == 0:
            continue
        
        for corte in np.arange(1, numero_cortes + 1):
            saida = prefixo_saida + "0-" + str(corte) + ".wav"
            
            # calcular trechos de início e fim no áudio
            inicio = PONTOS_ONDA * (corte - 1)
            termino = PONTOS_ONDA * corte
            ytc = yt[inicio:termino]
            
            # gravar arquivo de áudio em formato WAV
            sf.write(saida, ytc, sr, format='wav', subtype='PCM_16')

CPU times: user 41min 28s, sys: 1min 16s, total: 42min 45s
Wall time: 1h 10min 17s


## Calcular quantidade de arquivos gerados

In [21]:
arquivos = []
for especie in especies:
    arquivos.append(len(os.listdir(DIR_SAIDA + '/' + especie)))
#arquivos

In [28]:
df['arquivos_wav'] = arquivos
maior_qtde = max(df['arquivos_wav'])
df['diferenca'] = maior_qtde - df['arquivos_wav']
df['multiplicacao'] = df[['diferenca', 'arquivos_mp3']].apply(lambda x: np.round(x[0] / x[1], 0), axis=1)
df

,especie,arquivos_mp3,arquivos_wav,diferenca,multiplicacao
0,mimus_saturninus,148,1374,2266,15.0
1,ramphocelus_carbo,185,1220,2420,13.0
2,columba_livia,40,543,3097,77.0
3,turdus_leucomelas,322,2805,835,3.0
4,caracara_plancus,90,477,3163,35.0
5,megarynchus_pitangua,401,2640,1000,2.0
6,vanellus_chilensis,280,1624,2016,7.0
7,pitangus_sulphuratus,556,3640,0,0.0
8,turdus_rufiventris,209,2465,1175,6.0
9,furnarius_rufus,152,618,3022,20.0


## Gerar arquivos adicionais com deslocamento

In [65]:
a = [1,2,3,4,5,6,7,8,9,10]
a

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [66]:
np.roll(a,3)

array([ 8,  9, 10,  1,  2,  3,  4,  5,  6,  7])

In [73]:
shape = int(7.45 * SAMPLE_RATE)
shape

119200

In [76]:
int(shape / SAMPLE_RATE)

7

In [71]:
shape / 77

7740.25974025974

In [72]:
SAMPLE_RATE

16000

In [79]:
int(SAMPLE_RATE / 76)

210

In [81]:
int(SAMPLE_RATE / 76/2)*2

210

In [85]:
4200661 / 16000

262.5413125

In [89]:
%%time

for especie in ['columba_livia']:
#for especie in especies:
    print("especie:", especie)
    
    # multiplicação necessária para a espécie
    multiplicacao = int(df[df.especie == especie]['multiplicacao'].values[0])
    print("-> multiplicacao:", multiplicacao)
    if multiplicacao == 0:
        continue

    diferenca = int(df[df.especie == especie]['diferenca'].values[0])
    print("-> diferenca:", multiplicacao)

    qtde_arquivos = 0
#    for arquivo in os.listdir(os.path.join(DIR_ENTRADA, especie))[:1]:
#    for arquivo in os.listdir(os.path.join(DIR_ENTRADA, especie))[:20]:
    for arquivo in os.listdir(os.path.join(DIR_ENTRADA, especie)):

        # sair do laço se atingir a quantidade necessária
        if qtde_arquivos >= diferenca:
            break
            
        # montar nome do arquivo de entrada
        entrada = os.path.join(DIR_ENTRADA, especie, arquivo)
        if not entrada.endswith('.mp3'):
            continue
        #print("entrada:", entrada)

        prefixo_saida = os.path.join(DIR_SAIDA, especie, arquivo.replace('.mp3', '-'))
        #print("prefixo_saida:", prefixo_saida)
        
        # carregar áudio original em formato MP3
        y, sr = librosa.load(entrada, sr=SAMPLE_RATE, mono=True)
        numero_pontos = y.shape[0]
        print("-> numero_pontos:", numero_pontos)
        duracao = y.shape[0] / sr
        print("-> duracao:", duracao)

        # calcular o número de cortes a serem efetuados no áudio
        numero_cortes = round(duracao / DURACAO_TRECHO)
        #print("-> numero_cortes:", numero_cortes)
        if numero_cortes == 0:
            continue
        
        # calcular o número necessário de variações
        #maximo_variacoes = int(numero_pontos / SAMPLE_RATE)
        #print("-> maximo_variacoes:", maximo_variacoes)
        #numero_variacoes = int(numero_pontos / multiplicacao / SAMPLE_RATE) + 1
        numero_variacoes = int(multiplicacao / numero_cortes) + 1
        print("-> numero_variacoes:", numero_variacoes)
        
        #numero_variacoes = int(multiplicacao / numero_cortes) + 1
        #print("-> numero_variacoes:", numero_variacoes)
        #tamanho_offset = int(DURACAO_TRECHO * PONTOS_ONDA / (numero_variacoes + 1))
        #print("-> tamanho_offset:", tamanho_offset)
        
        for variacao in np.arange(1, numero_variacoes + 1):
            
            # sair do laço se atingir a quantidade necessária
            if qtde_arquivos >= diferenca:
                break

            tamanho_roll = int(SAMPLE_RATE / 10 * variacao)
            print("--> tamanho_roll:", tamanho_roll)
            yo = np.roll(y, tamanho_roll)
            
            # deslocar o áudio para a direita
            #offset = tamanho_offset * variacao
            #print("-> offset:", offset)
            #yo = y[offset:]

            for corte in np.arange(1, numero_cortes + 1):

                # sair do laço se atingir a quantidade necessária
                if qtde_arquivos >= diferenca:
                    break

                saida = prefixo_saida + str(variacao) + "-" + str(corte) + ".wav"
                    
                # remover silêncio nas extremidades do áudio
                yt = yo
                #yt, index = librosa.effects.trim(yo)
                duracao = yt.shape[0] / sr
                
                if (duracao / DURACAO_TRECHO < 0.3):
                    break
                
                # calcular trechos de início e fim no áudio
                inicio = PONTOS_ONDA * (corte - 1)
                termino = PONTOS_ONDA * corte
                ytc = yt[inicio:termino]
                
                duracao = ytc.shape[0] / sr
                if (duracao / DURACAO_TRECHO < 0.3):
                    break

                # gravar arquivo de áudio em formato WAV
                sf.write(saida, ytc, sr, format='wav', subtype='PCM_16')
                qtde_arquivos += 1
                
    print("-> qtde_arquivos:", qtde_arquivos, "\n")

especie: columba_livia
-> multiplicacao: 77
-> diferenca: 77
-> numero_pontos: 4200661
-> duracao: 262.5413125
-> numero_variacoes: 2
--> tamanho_roll: 1600
--> tamanho_roll: 3200
-> numero_pontos: 546691
-> duracao: 34.1681875
-> numero_variacoes: 12
--> tamanho_roll: 1600
--> tamanho_roll: 3200
--> tamanho_roll: 4800
--> tamanho_roll: 6400
--> tamanho_roll: 8000
--> tamanho_roll: 9600
--> tamanho_roll: 11200
--> tamanho_roll: 12800
--> tamanho_roll: 14400
--> tamanho_roll: 16000
--> tamanho_roll: 17600
--> tamanho_roll: 19200
-> numero_pontos: 4310451
-> duracao: 269.4031875
-> numero_variacoes: 2
--> tamanho_roll: 1600
--> tamanho_roll: 3200
-> numero_pontos: 168020
-> duracao: 10.50125
-> numero_variacoes: 39
--> tamanho_roll: 1600
--> tamanho_roll: 3200
--> tamanho_roll: 4800
--> tamanho_roll: 6400
--> tamanho_roll: 8000
--> tamanho_roll: 9600
--> tamanho_roll: 11200
--> tamanho_roll: 12800
--> tamanho_roll: 14400
--> tamanho_roll: 16000
--> tamanho_roll: 17600
--> tamanho_roll: 1

-> numero_pontos: 2586215
-> duracao: 161.6384375
-> numero_variacoes: 3
--> tamanho_roll: 1600
--> tamanho_roll: 3200
--> tamanho_roll: 4800
-> numero_pontos: 262736
-> duracao: 16.421
-> numero_variacoes: 26
--> tamanho_roll: 1600
--> tamanho_roll: 3200
--> tamanho_roll: 4800
--> tamanho_roll: 6400
--> tamanho_roll: 8000
--> tamanho_roll: 9600
--> tamanho_roll: 11200
--> tamanho_roll: 12800
--> tamanho_roll: 14400
--> tamanho_roll: 16000
--> tamanho_roll: 17600
--> tamanho_roll: 19200
--> tamanho_roll: 20800
--> tamanho_roll: 22400
--> tamanho_roll: 24000
--> tamanho_roll: 25600
--> tamanho_roll: 27200
--> tamanho_roll: 28800
--> tamanho_roll: 30400
--> tamanho_roll: 32000
--> tamanho_roll: 33600
--> tamanho_roll: 35200
--> tamanho_roll: 36800
--> tamanho_roll: 38400
--> tamanho_roll: 40000
--> tamanho_roll: 41600
-> numero_pontos: 1846962
-> duracao: 115.435125
-> numero_variacoes: 4
--> tamanho_roll: 1600
--> tamanho_roll: 3200
--> tamanho_roll: 4800
--> tamanho_roll: 6400
-> numer

--> tamanho_roll: 12800
--> tamanho_roll: 14400
-> numero_pontos: 348831
-> duracao: 21.8019375
-> numero_variacoes: 20
--> tamanho_roll: 1600
--> tamanho_roll: 3200
--> tamanho_roll: 4800
--> tamanho_roll: 6400
--> tamanho_roll: 8000
--> tamanho_roll: 9600
--> tamanho_roll: 11200
--> tamanho_roll: 12800
--> tamanho_roll: 14400
--> tamanho_roll: 16000
--> tamanho_roll: 17600
--> tamanho_roll: 19200
--> tamanho_roll: 20800
--> tamanho_roll: 22400
--> tamanho_roll: 24000
--> tamanho_roll: 25600
--> tamanho_roll: 27200
--> tamanho_roll: 28800
--> tamanho_roll: 30400
--> tamanho_roll: 32000
-> numero_pontos: 334080
-> duracao: 20.88
-> numero_variacoes: 20
--> tamanho_roll: 1600
--> tamanho_roll: 3200
--> tamanho_roll: 4800
--> tamanho_roll: 6400
--> tamanho_roll: 8000
--> tamanho_roll: 9600
--> tamanho_roll: 11200
--> tamanho_roll: 12800
--> tamanho_roll: 14400
--> tamanho_roll: 16000
--> tamanho_roll: 17600
--> tamanho_roll: 19200
--> tamanho_roll: 20800
--> tamanho_roll: 22400
--> taman

## Calcular quantidade de arquivos gerados ao final

In [ ]:
arquivos = []
for especie in especies:
    arquivos.append(len(os.listdir(DIR_SAIDA + '/' + especie)))
#arquivos

In [ ]:
df['arquivos_wav2'] = arquivos
df